In [ ]:
print("hello world")

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from torchvision.transforms import Compose, ToTensor,ToPILImage, Normalize, Resize, CenterCrop, RandomResizedCrop
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3
from torch.hub import load_state_dict_from_url
from stepbystep.v3 import StepByStep

## Pre-Trained Model

In [ ]:
alex = alexnet(weights=None)
print(alex)

In [ ]:
from torchinfo import summary
summary(alex,input_size=(16,3,224,224))

### Average Pooling  
Returns the tensor with desired shape regardless of the input shape

In [ ]:
result1=F.adaptive_avg_pool2d(input=torch.randn(16,32,32),output_size=(6,6))
result2=F.adaptive_avg_pool2d(input=torch.randn(16,32,32),output_size=(6,6))
result1.shape,result2.shape

### Loading Weights

In [ ]:
# URL for alexnet pretrained weights


In [ ]:
from torchvision.models import AlexNet_Weights
alex=alexnet(weights=AlexNet_Weights.DEFAULT)

In [ ]:
print(alex)

In [ ]:
# Model freezing
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad=False

In [ ]:
freeze_model(alex)

### Top of the model

In [ ]:
alex.classifier

In [ ]:
#replacing the top of the model
alex.classifier[6]=nn.Linear(4096,3)

In [ ]:
# Check for gradient calculation requirement of output layer
for name,param in alex.named_parameters():
    if param.requires_grad==True:
        print(name)

In [ ]:
summary(alex,input_size=(1,3,224,224))

In [ ]:
# Model config
torch.manual_seed(17)
multi_loss_fn=nn.CrossEntropyLoss(reduction='mean')
optimizer_alex=optim.Adam(alex.parameters(),lr=3e-4)

In [ ]:
# Data prep
normalizer=Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
composer=Compose(transforms=[Resize(256),CenterCrop(224),ToTensor(),normalizer])

train_data=ImageFolder(root='rps',transform=composer)
val_data=ImageFolder(root='rps-test-set',transform=composer)

train_loader=DataLoader(dataset=train_data,batch_size=16,shuffle=True)
val_loader=DataLoader(dataset=val_data,batch_size=16)

In [ ]:
# Model Training
sbs_alex=StepByStep(alex,multi_loss_fn,optimizer_alex)
sbs_alex.set_loaders(train_loader,val_loader)
sbs_alex.train(10)